In [43]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical

In [44]:
df = pd.read_csv("matches.csv") 


data = df[['team1', 'team2', 'toss_winner']].dropna()

data

,team1,team2,toss_winner
0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore
1,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings
2,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals
3,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians
4,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers
...,...,...,...
1090,Punjab Kings,Sunrisers Hyderabad,Punjab Kings
1091,Sunrisers Hyderabad,Kolkata Knight Riders,Sunrisers Hyderabad
1092,Royal Challengers Bengaluru,Rajasthan Royals,Rajasthan Royals
1093,Sunrisers Hyderabad,Rajasthan Royals,Rajasthan Royals


In [45]:
label_encoders = {}
for col in ['team1', 'team2', 'toss_winner']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col].astype(str)) 
    label_encoders[col] = le 

In [ ]:
X = data[['team1', 'team2']].astype(np.float32)  # Convert to float32
y = data['toss_winner'].astype(np.int32)  # Convert to int32
# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

num_classes = len(label_encoders['toss_winner'].classes_)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Defining the neural network model
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    
    keras.layers.Dense(len(label_encoders['toss_winner'].classes_), activation='softmax')  # Multi-class classification
])

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(X_train, y_train, epochs=1000, batch_size=16, validation_data=(X_test, y_test))



Epoch 1/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.1157 - loss: 2.8468 - val_accuracy: 0.2009 - val_loss: 2.6753
Epoch 2/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1695 - loss: 2.5806 - val_accuracy: 0.2420 - val_loss: 2.5626
Epoch 3/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1789 - loss: 2.5392 - val_accuracy: 0.2420 - val_loss: 2.4118
Epoch 4/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2005 - loss: 2.3514 - val_accuracy: 0.2557 - val_loss: 2.4043
Epoch 5/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1954 - loss: 2.3762 - val_accuracy: 0.2511 - val_loss: 2.2837
Epoch 6/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1952 - loss: 2.3859 - val_accuracy: 0.2100 - val_loss: 2.3034
Epoch 7/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2103 - loss: 2.3571 - val_accuracy: 0.3059 - val_loss: 2.2750
Epoch 8/1000
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2363 - loss: 2.2821 - val_accuracy: 0

In [47]:
# Evaluating the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5328 - loss: 1.3290 
Test Accuracy: 50.68%


In [49]:
def predict_toss_winner(team1, team2):
    team1_encoded = label_encoders['team1'].transform([team1])[0]
    team2_encoded = label_encoders['team2'].transform([team2])[0]
    
    
    input_data = np.array([[team1_encoded, team2_encoded]])
    
    predictions = model.predict(input_data)
    
    predicted_winner_index = np.argmax(predictions)
    
    predicted_winner = label_encoders['toss_winner'].inverse_transform([predicted_winner_index])[0]
    
    return predicted_winner

team1 = "Kolkata Knight Riders"
team2 = "Royal Challengers Bangalore"

predicted_winner = predict_toss_winner(team1, team2)
print(f"Predicted Toss Winner: {predicted_winner}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted Toss Winner: Kolkata Knight Riders
